In [23]:
import numpy as np
from collections import Counter, defaultdict
import re
from tqdm.auto import tqdm
from typing import List, Union, Tuple


class PrefixTreeNode:
    """
    Узел для префиксного дерева.
    Хранит детей узла и флаг конца слова.
    """
    def __init__(self):
        self.children = {}  # Дочерние узлы
        self.is_end_of_word = False  # Является ли конец слова


class PrefixTree:
    """
    Префиксное дерево (трис) для хранения слов и поиска по префиксам.
    """
    def __init__(self, vocabulary: List[str]):
        self.root = PrefixTreeNode()
        for word in vocabulary:
            self.insert(word)

    def insert(self, word: str):
        """
        Добавляет слово в префиксное дерево.

        :param word: Слово для вставки
        """
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = PrefixTreeNode()
            node = node.children[char]
        node.is_end_of_word = True

    def _collect_words(self, node: PrefixTreeNode, prefix: str) -> List[str]:
        """
        Рекурсивно собирает слова, которые начинаются с заданного префикса.

        :param node: Текущий узел дерева
        :param prefix: Текущий префикс
        :return: Список слов
        """
        words = []
        if node.is_end_of_word:
            words.append(prefix)
        for char, child_node in node.children.items():
            words.extend(self._collect_words(child_node, prefix + char))
        return words

    def search_prefix(self, prefix: str) -> List[str]:
        """
        Находит все слова, начинающиеся с данного префикса.

        :param prefix: Префикс для поиска
        :return: Список слов, начинающихся с префикса
        """
        node = self.root
        for char in prefix:
            if char in node.children:
                node = node.children[char]
            else:
                return []
        return self._collect_words(node, prefix)


class WordCompletor:
    """
    Класс для дополнения слов на основе частоты их использования и префиксного дерева.
    """
    def __init__(self, corpus: List[List[str]] = None, word_freqs: Counter = None, min_wf: int = 5):
        """
        Инициализирует WordCompletor.

        :param corpus: Корпус текстов для подсчета частоты слов
        :param word_freqs: Готовые частоты слов (если есть)
        :param min_wf: Минимальная частота слова для включения
        """
        if word_freqs is not None:
            self.word_freqs = word_freqs
        else:
            self.word_freqs = Counter()
            for text in tqdm(corpus, desc="Calculating word frequencies"):
                self.word_freqs.update(Counter(text))

            # Удаление слов с частотой меньше min_wf
            self.word_freqs = {word: count for word, count in self.word_freqs.items() if count >= min_wf}

            # Нормализация частот
            total = sum(self.word_freqs.values())
            for k in self.word_freqs:
                self.word_freqs[k] /= total

        self.prefix_tree = PrefixTree(self.word_freqs.keys())

    def get_words_and_probs(self, prefix: str) -> Tuple[List[str], List[float]]:
        """
        Возвращает слова и их вероятности для заданного префикса.

        :param prefix: Префикс для поиска
        :return: Список слов и их вероятности
        """
        words = self.prefix_tree.search_prefix(prefix)
        if not words:
            return [], []
        probs = [self.word_freqs[word] for word in words]
        return words, probs


class NGramLanguageModel:
    """
    N-граммная языковая модель для предсказания следующего слова.
    """
    def __init__(self, corpus: List[List[str]] = None, n_gram_freq: defaultdict = None, n: int = 3, min_wf: int = 4):
        """
        Инициализирует NGramLanguageModel.

        :param corpus: Корпус текстов для подсчета n-грамм
        :param n_gram_freq: Частоты n-грамм (если есть)
        :param n: Длина n-грамм
        :param min_wf: Минимальная частота для включения
        """
        self.n = n
        self.min_wf = min_wf
        self.n_gram_freq = n_gram_freq or defaultdict(Counter)
        if corpus:
            self.count_ngrams(corpus)

    def count_ngrams(self, corpus: List[List[str]]):
        """
        Считает частоты n-грамм в корпусе.

        :param corpus: Корпус текстов
        """
        for line in tqdm(corpus, desc="Counting n-grams"):
            for i in range(self.n, len(line)):
                prefix = tuple(line[i-self.n:i])
                token = line[i]
                self.n_gram_freq[prefix][token] += 1

        # Фильтрация редких n-грамм и расчет вероятностей
        for prefix in list(self.n_gram_freq.keys()):
            token_counter = self.n_gram_freq[prefix]
            filtered_tokens = {word: count for word, count in token_counter.items() if count >= self.min_wf}
            if filtered_tokens:
                total_count = sum(filtered_tokens.values())
                self.n_gram_freq[prefix] = {word: count / total_count for word, count in filtered_tokens.items()}
            else:
                del self.n_gram_freq[prefix]

    def get_last_n_tokens(self, prefix: List[str]) -> Tuple[str, ...]:
        """
        Возвращает последние n токенов из списка.

        :param prefix: Список токенов
        :return: Кортеж последних n токенов
        """
        return tuple(prefix[-self.n:])

    def get_next_words_and_probs(self, prefix: List[str]) -> Tuple[List[str], List[float]]:
        """
        Возвращает слова и их вероятности для продолжения префикса.

        :param prefix: Префикс для поиска
        :return: Список слов и их вероятности
        """
        prefix = self.get_last_n_tokens(prefix)
        if prefix in self.n_gram_freq:
            possible_tokens = self.n_gram_freq[prefix]
            next_words = list(possible_tokens.keys())
            probs = list(possible_tokens.values())
            return next_words, probs
        return [], []


class TextSuggestion:
    """
    Класс для предложения текстовых дополнений на основе модели.
    """
    def __init__(self, word_completor: WordCompletor, n_gram_model: NGramLanguageModel):
        self.word_completor = word_completor
        self.n_gram_model = n_gram_model

    @staticmethod
    def tokenize(text: str) -> List[str]:
        """
        Токенизирует текст.

        :param text: Исходный текст
        :return: Список токенов
        """
        reg = re.compile(r'\w+')
        return reg.findall(text.lower())

    def suggest_text(self, text: Union[str, List[str]], n_words: int = 3) -> List[str]:
        """
        Предлагает продолжение текста.

        :param text: Исходный текст или список токенов
        :param n_words: Количество слов для предложения
        :return: Список предложенных слов
        """
        if isinstance(text, str):
            text = self.tokenize(text)

        suggestions = []
        last_text_token = text[-1]

        words, probs = self.word_completor.get_words_and_probs(last_text_token)
        if words:
            max_index = np.argmax(probs)
            key_word = words[max_index]
            suggestions.append(key_word)
            text[-1] = key_word

            for _ in range(n_words):
                words, probs = self.n_gram_model.get_next_words_and_probs(text)
                if words:
                    max_index = np.argmax(probs)
                    key_word = words[max_index]
                    text.append(key_word)
                    suggestions.append(key_word)
                else:
                    break

        return suggestions


def main():
    """
    Основная функция для демонстрации работы классов.
    """
    # Загрузка данных
    with open('/kaggle/input/emails/emails.txt', 'r') as f:
        emails = [text.split() for text in f.read().split('\n')]

    # Инициализация классов
    word_completor = WordCompletor(emails, min_wf=5)
    n_gram_model = NGramLanguageModel(corpus=emails, n=3, min_wf=4)
    text_suggestion = TextSuggestion(word_completor, n_gram_model)

    # Пример использования
    input_text = "can we speed up t"
    suggestions = text_suggestion.suggest_text(input_text)
    print("Предложения:", suggestions)


if __name__ == '__main__':
    main()


Calculating word frequencies:   0%|          | 0/308361 [00:00<?, ?it/s]

Counting n-grams:   0%|          | 0/308361 [00:00<?, ?it/s]

Предложения: ['the', 'process', 'of', 'getting']
